In [1]:
import pandas as pd
import os
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
# Load the CSV file into a Pandas DataFrame.
df = pd.read_csv('dataset/HAM10000_metadata.csv')

# Extract the image paths and labels from the DataFrame.
image_paths = df['image_id'].tolist()
labels = df['dx'].tolist()

In [3]:
# Create a list of image file paths.
image_file_paths = []
for image_path in image_paths:
    image_file_path = os.path.join('images', image_path + '.jpg')
    image_file_paths.append(image_file_path)


In [4]:
# Create a TensorFlow dataset.
dataset = tf.data.Dataset.from_tensor_slices((image_file_paths, labels))

# Load the images from the file paths.
dataset = dataset.map(lambda image_file_path, label: (tf.image.decode_jpeg(tf.io.read_file(image_file_path)), label))

# Resize the images to 224x224 pixels.
dataset = dataset.map(lambda image, label: (tf.image.resize(image, (224, 224)), label))

# Normalize the pixel values to the range [0, 1].
dataset = dataset.map(lambda image, label: (image / 255.0, label))

# Split the dataset into training and test sets.
train_dataset = dataset.take(int(0.8 * len(dataset)))
test_dataset = dataset.skip(int(0.8 * len(dataset)))

# Create TensorFlow batches from the training and test sets.
train_batch_size = 32
test_batch_size = 32

train_dataset = train_dataset.batch(train_batch_size)
test_dataset = test_dataset.batch(test_batch_size)

In [5]:
# Load the ResNet50 model with pre-trained weights.
model = ResNet50(weights='imagenet')

# Freeze the pre-trained weights so that they are not updated during training.
for layer in model.layers[:175]:
    layer.trainable = False

In [6]:
# Add a new fully connected layer with 7 outputs, corresponding to the 7 classes in the HAM10000 dataset.
model.add(tf.keras.layers.Dense(7, activation='softmax'))

AttributeError: 'Functional' object has no attribute 'add'

: 